# Deep Convolutional Generative Adversarial Network (DCGAN)

A tutorial for GAN image generation with DCGAN architecture. Adapting https://github.com/ravasconcelos/flowers_dcgan

### Mounting Google Drive

In [ ]:
from google.colab import drive  # Mount Drive to access our images

drive.mount("/content/gdrive", force_remount=True)

### Configuration

Settings for our project. 

- Image size is 256px X 256px
- Epochs = the number of "rounds" this dataset is being studied by the model
- Batch size = number of samples processed before model gets updated
- Noise size = parameter for the image generator, needs random noise as input to produce new samples
- Number of new images created is 1000
- Path to our dataset folder

In [ ]:
IMAGE_SIZE = 256
EPOCHS = 200
BATCH_SIZE = 64
NOISE_SIZE = 100
NUM_NEW_IMAGES = 1000
DATASET_FOLDER = "/content/gdrive/My Drive/Colab Notebooks/CTL_DCGAN/cats/"

### Importing libraries.

In [ ]:
from datetime import datetime
import os
from glob import glob
from IPython import display
import imageio
import matplotlib.pyplot as plt

%matplotlib inline
import numpy as np
import PIL
from PIL import Image
import pytz

tz_NY = pytz.timezone("America/New_York")
import random
from scipy import ndarray
import skimage as sk
from skimage import io
from skimage import util
from skimage import transform
import tensorflow as tf
from tensorflow.keras import layers
import time

### Defining models

Both the generator and discriminator are defined using the [Keras Sequential API](https://www.tensorflow.org/guide/keras#sequential_model).

### Generator

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(64 * 64 * 256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((64, 64, 256)))
    assert model.output_shape == (None, 64, 64, 256)  # Note: None is the batch size

    model.add(
        layers.Conv2DTranspose(
            128, (5, 5), strides=(1, 1), padding="same", use_bias=False
        )
    )
    assert model.output_shape == (None, 64, 64, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(
        layers.Conv2DTranspose(
            64, (5, 5), strides=(2, 2), padding="same", use_bias=False
        )
    )
    assert model.output_shape == (None, 128, 128, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(
        layers.Conv2DTranspose(
            3, (5, 5), strides=(2, 2), padding="same", use_bias=False, activation="tanh"
        )
    )
    assert model.output_shape == (None, 256, 256, 3)

    return model


generator = make_generator_model()

# Use the (as yet untrained) generator to create an image.
noise = tf.random.normal([1, NOISE_SIZE])
generated_image = generator(noise, training=False)

generated_image2 = generated_image[0].numpy() * 127.5 + 127.5

plt.imshow(generated_image2.astype("uint8"))

### Discriminator

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(
        layers.Conv2D(
            64, (5, 5), strides=(2, 2), padding="same", input_shape=[256, 256, 3]
        )
    )
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding="same"))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model


discriminator = make_discriminator_model()
# Use the (as yet untrained) discriminator to classify the generated images as real or fake.
# The model will be trained to output positive values for real images, and negative values for fake images.
decision = discriminator(generated_image)
print(decision)

### Data Preparation

Here we make some functions for displaying our images and learning progress. These will be used later in the notebook.

In [ ]:
def show_samples(sample_images):
    print("len(sample_images): ", len(sample_images))
    print("len(sample_images): ", sample_images[0].shape)

    figure, axes = plt.subplots(1, len(sample_images), figsize=(50, 50))

    print("figure: ", figure)
    print("axes: ", axes)

    for index, axis in enumerate(axes):
        axis.axis("off")
        image_array = sample_images[index]
        axis.imshow(image_array)

    plt.show()
    plt.close()

In [ ]:
def show_image_custom(input_image):
    fig = plt.figure(figsize=(4, 4))

    plt.imshow(input_image)

    plt.axis("off")

    plt.show()

In [ ]:
def summarize_epoch(epoch, d_losses, g_losses, save_image=True):
    fig, ax = plt.subplots()
    plt.plot(d_losses, label="Discriminator", alpha=0.6)
    plt.plot(g_losses, label="Generator", alpha=0.6)
    plt.title("Losses")
    plt.legend()
    if save_image:
        plt.savefig("losses_" + str(epoch) + ".png")
    plt.show()
    plt.close()

In [ ]:
def generate_and_save_images(model, epoch, test_input, save_image=True):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        generated_image2 = predictions[i].numpy() * 127.5 + 127.5
        plt.imshow(generated_image2.astype("uint8"))

        plt.axis("off")

    if save_image:
        plt.savefig("image_at_epoch_{:04d}.png".format(epoch))
    plt.show()

In [ ]:
def show_samples2(sample_images):
    figure, axes = plt.subplots(1, len(sample_images), figsize=(50, 50))

    for index, axis in enumerate(axes):
        axis.axis("off")
        image_array = sample_images[index]
        image_array = image_array.numpy() * 127.5 + 127.5
        axis.imshow(image_array.astype(np.uint8))

    plt.show()
    plt.close()

### Data Augmentation

Image transformation methods. These are functions that we use later to our images. The goal is to "see" our dataset from different angles: imagine that the ML model is walking around the cat and closer to it instead of just watching from one distance and angle :)

In [ ]:
def random_rotation(image_array: ndarray):
    # pick a random degree of rotation between 20% on the left and 20% on the right
    random_degree = random.uniform(-20, 20)
    return sk.transform.rotate(image_array, random_degree)


def horizontal_flip(image_array: ndarray):
    # horizontal flip doesn't need skimage, it's easy as flipping the image array of pixels !
    return image_array[:, ::-1]


def vertical_flip(image_array: ndarray):
    return image_array[::-1, :]


def vertical_and_horizontal_flip(image_array: ndarray):
    h_flip_image = vertical_flip(image_array)
    return horizontal_flip(h_flip_image)


def TF_crop_pad(x, n_pixels=20, pad_mode="edge"):
    """Pad image by n_pixels on each size, then take random crop of same
    original size.
    """
    assert len(x.shape) == 3
    h, w, nc = x.shape

    # First pad image by n_pixels on each side
    padded = sk.util.pad(
        x, [(n_pixels, n_pixels) for _ in range(2)] + [(0, 0)], mode=pad_mode
    )

    # Then take a random crop of the original size
    crops = [(c, 2 * n_pixels - c) for c in np.random.randint(0, 2 * n_pixels + 1, [2])]
    # For channel dimension don't do any cropping
    crops += [(0, 0)]
    return sk.transform.resize(
        sk.util.crop(padded, crops, copy=True), (IMAGE_SIZE, IMAGE_SIZE)
    )

Create these new images for better learning. We do this by doing random transformation on random images.

In [ ]:
# dictionary of the transformations we defined earlier
available_transformations = {
    "rotate": random_rotation,
    "vertical_flip": vertical_flip,
    "horizontal_flip": horizontal_flip,
    "vertical_and_horizontal_flip": vertical_and_horizontal_flip,
    "TF_crop_pad": TF_crop_pad,
}

folder_path = DATASET_FOLDER

# find all files paths from the folder
images = [
    os.path.join(folder_path, f)
    for f in os.listdir(folder_path)
    if os.path.isfile(os.path.join(folder_path, f))
]

num_generated_files = 0
while num_generated_files <= NUM_NEW_IMAGES:
    # random image from the folder
    image_path = random.choice(images)
    # read image as an two dimensional array of pixels
    image_to_transform = sk.io.imread(image_path)
    # random num of transformation to apply
    num_transformations_to_apply = random.randint(1, len(available_transformations))

    num_transformations = 0
    transformed_image = None
    while num_transformations <= num_transformations_to_apply:
        # random transformation to apply for a single image
        key = random.choice(list(available_transformations))
        transformed_image = available_transformations[key](image_to_transform)
        num_transformations += 1

        new_file_path = "%s/augmented_image_%s.jpg" % (folder_path, num_generated_files)

        # write image to the disk
        # io.imsave(new_file_path, transformed_image.astype(np.uint8))
        io.imsave(new_file_path, transformed_image)
        num_generated_files += 1

### Load and prepare your custom dataset + show samples of it.

In [ ]:
input_images = np.asarray(
    [
        np.asarray(Image.open(file).resize((IMAGE_SIZE, IMAGE_SIZE)))
        for file in glob(DATASET_FOLDER + "*")
    ]
)
print("Input: " + str(input_images.shape))

np.random.shuffle(input_images)

train_images = input_images.reshape(input_images.shape[0], 256, 256, 3).astype(
    "float32"
)
train_images = (train_images - 127.5) / 127.5

BUFFER_SIZE = input_images.shape[0]
# Batch and shuffle the data
train_dataset = (
    tf.data.Dataset.from_tensor_slices(train_images)
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
)

sample_images = input_images[:5]
show_samples(sample_images)

Show one image and the transformations - remember, this is just preparation, for the model to see the dataset from different angles.

In [ ]:
show_image_custom(input_images[0])

In [ ]:
show_image_custom(vertical_flip(input_images[0]))

In [ ]:
show_image_custom(horizontal_flip(input_images[0]))

In [ ]:
show_image_custom(random_rotation(input_images[0]))

In [ ]:
show_image_custom(vertical_and_horizontal_flip(input_images[0]))

### 8) Loss and optimizers

Define loss functions and optimizers for both models.

In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

**Discriminator loss**. This method quantifies how well the discriminator is able to distinguish real images from fakes. It compares the discriminator's predictions on real images to an array of 1s, and the discriminator's predictions on fake (generated) images to an array of 0s.

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

**Generator loss**. The generator's loss quantifies how well it was able to trick the discriminator. Intuitively, if the generator is performing well, the discriminator will classify the fake images as real (or 1). Here, we will compare the discriminators decisions on the generated images to an array of 1s.

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

The discriminator and the generator optimizers are different since we will train two networks separately.

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

### Training functions


**Save checkpoints**. This means saving the current status of the training. Can be helpful in case a long running training task is interrupted.

In [ ]:
checkpoint_dir = "./training_checkpoints"
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(
    generator_optimizer=generator_optimizer,
    discriminator_optimizer=discriminator_optimizer,
    generator=generator,
    discriminator=discriminator,
)

**Noise sample array** - preparations for giving the generator model some random noise as input.

In [ ]:
noise_dim = NOISE_SIZE
num_examples_to_generate = 16

# We will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

**The training loop** begins with generator receiving a random seed as input. That seed is used to produce an image. The discriminator is then used to classify real images (drawn from the training set) and fakes images (produced by the generator). The loss is calculated for each of these models, and the gradients are used to update the generator and discriminator.

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(
        disc_loss, discriminator.trainable_variables
    )

    generator_optimizer.apply_gradients(
        zip(gradients_of_generator, generator.trainable_variables)
    )
    discriminator_optimizer.apply_gradients(
        zip(gradients_of_discriminator, discriminator.trainable_variables)
    )

    return gen_loss, disc_loss

In [ ]:
def train(dataset, epochs):
    print("Training started at: ", datetime.now(tz_NY))
    save_image = False
    d_losses = []
    g_losses = []
    for epoch in range(epochs):
        start = time.time()

        for image_batch in dataset:
            d_loss, g_loss = train_step(image_batch)
            d_losses.append(d_loss)
            g_losses.append(g_loss)

        # Produce images for the GIF as we go
        display.clear_output(wait=True)

        # Save the model every 20 epochs
        if (epoch + 1) % 20 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)
            save_image = True

        generate_and_save_images(generator, epoch + 1, seed, save_image)
        summarize_epoch(epoch, d_losses, g_losses, save_image)
        save_image = False
        print("Time for epoch {} is {} sec".format(epoch + 1, time.time() - start))

    # Generate after the final epoch
    display.clear_output(wait=True)
    generate_and_save_images(generator, epochs, seed)
    summarize_epoch(epoch, d_losses, g_losses)

### Finally, training the model!
Call the `train()` method defined above to train the generator and discriminator simultaneously. How long this takes depends on many things - mainly your dataset size and number of epochs.

In [ ]:
train(train_dataset, EPOCHS)

In [ ]:
checkpoint.save(file_prefix=checkpoint_prefix)

To restore a checkpoint upload a the ckpt files into training_checkpoints folder.

In [ ]:
# upload the files (checkpoint, ckpt-xxx.index, ckpt-xxx.data-*) into training_checkpoints folder
RESTORE_CHECKPOINT = True
if RESTORE_CHECKPOINT:
    checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

### Generate some images!

In [ ]:
noise = tf.random.normal([2, NOISE_SIZE])
generated_image = generator(noise, training=False)
show_samples2(generated_image)

In [ ]:
noise = tf.random.normal([5, NOISE_SIZE])
generated_image = generator(noise, training=False)
show_samples2(generated_image)

### Create a GIF


The GIF uses images from different steps in the training. Thus, it visualises how the model has learned.

In [ ]:
# Display a single image using the epoch number
def display_image(epoch_no):
    return PIL.Image.open("image_at_epoch_{:04d}.png".format(epoch_no))

In [ ]:
display_image(EPOCHS)

Create an animated gif using the images saved during training.

In [ ]:
anim_file = "dcgan.gif"

with imageio.get_writer(anim_file, mode="I") as writer:
    filenames = glob("image*.png")
    filenames = sorted(filenames)
    last = -1
    for i, filename in enumerate(filenames):
        frame = 2 * (i**0.5)
        if round(frame) > round(last):
            last = frame
        else:
            continue
        image = imageio.imread(filename)
        writer.append_data(image)
    image = imageio.imread(filename)
    writer.append_data(image)

import IPython

if IPython.version_info > (6, 2, 0, ""):
    display.Image(filename=anim_file)

You can download the animation with the code below:

In [ ]:
try:
    from google.colab import files
except ImportError:
    pass
else:
    files.download(anim_file)